In [ ]:
import pandas as pd
import numpy as np
import spacy
import string
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,classification_report
import gensim.downloader as api

In [ ]:
review=pd.read_csv("dev_with_labels.tsv",delimiter="\t")
review=review.rename(columns={"Text data": "data"},inplace=False)


In [ ]:
#load spacy
nlp=spacy.load("en_core_web_sm")
stop_words=nlp.Defaults.stop_words
punctuations=string.punctuation

In [ ]:
wv=api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
def spacy_tokenizer(sentence):
    doc=nlp(sentence)
    mytokens = [word.lemma_.lower().strip() for word in doc]
    mytokens = [word for word in mytokens if word not in stop_words and word not in punctuations]
    return mytokens


In [ ]:
review["data1"]=review["data"].apply(spacy_tokenizer)    #tokenizing text data

In [ ]:
review['vec'] = review['data1'].apply(lambda x: np.mean([wv[token] for token in x if token in wv] or [np.zeros(wv.vector_size)], axis=0))

In [ ]:
#split dataset
x=np.vstack(review["vec"])
y=review["Label"]
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.8,random_state=1)

In [ ]:
# Define the model pipeline with Logistic Regression
model_pipeline_lr = Pipeline([
    ('lr', LogisticRegression(max_iter=1000))  # Increase max_iter to 1000 or more
])

# Train the model
model_pipeline_lr.fit(x_train, y_train)

# Predict on the test set
y_pred_lr = model_pipeline_lr.predict(x_test)

In [ ]:
# Evaluate the model
accuracy_lr = accuracy_score(y_test, y_pred_lr)
precision_lr = precision_score(y_test, y_pred_lr, average='weighted')
recall_lr = recall_score(y_test, y_pred_lr, average='weighted')
f1_score_lr = f1_score(y_test, y_pred_lr, average='weighted')
classification_report_lr = classification_report(y_test, y_pred_lr)

# Print evaluation metrics
print("Evaluation Metrics for Logistic Regression Model")
print("------------------------------------------------")
print(classification_report_lr)

Evaluation Metrics for Logistic Regression Model
------------------------------------------------
                precision    recall  f1-score   support

      moderate       0.56      0.83      0.67       455
not depression       0.60      0.35      0.45       363
        severe       0.75      0.11      0.19        82

      accuracy                           0.57       900
     macro avg       0.64      0.43      0.44       900
  weighted avg       0.59      0.57      0.53       900

